In [1]:
from funcs_ssp_study import *

import os
import xarray as xr
import pandas as pd
import geopandas as gpd
import re
import json
from tqdm import tqdm
import cdsapi
import zipfile
import shutil
import requests

# Code

In [7]:
data_directory = "/Users/michellewan/PycharmProjects/TraCE/data_pipeline"
data_subdirectory = os.path.join(data_directory, "raw")
output_subdirectory = os.path.join(data_directory, "processed_monthly")

country_shapes_filepath = "/Users/michellewan/PycharmProjects/TraCE/data_pipeline/ne_10m_admin_0_countries/ne_10m_admin_0_countries.shp"
country_codes_filepath = "/Users/michellewan/PycharmProjects/TraCE/data_pipeline/OECD_population_GDP/OECD_country_codes.csv"
alignments_filepath = "/Users/michellewan/PycharmProjects/TraCE/alignments.json"

socioeconomic_directory = "/Users/michellewan/Downloads/OECD_population_GDP"
socioeconomic_ssp_filepath = os.path.join(socioeconomic_directory, "SspDb_country_data_2013-06-12.csv")
population_historical_filepath = os.path.join(socioeconomic_directory, "HISTPOP_22082023151824127.csv")
gdp_historical_filepath = os.path.join(socioeconomic_directory, "DP_LIVE_22082023165117703.csv")

In [3]:
climate_variables = [
    "temperature", 
    "pressure",
    "wind_speed",
    "precipitation"
]

socioeconomic_variables = [
    "GDP|PPP", 
    "Population"
]

atmospheric_variables = [
    "ch4"
]

In [4]:
SSPs = [str(ssp) for ssp in range(1, 5+1)]

In [5]:
frequency = "M"

start_year = 2015
end_year = 2023

## DOWNLOAD

In [ ]:
# Downloading climate data from Copernicus CDS requires a registered account (free)
download_climate_variables_data(data_subdirectory, climate_variables, SSPs, start_year, end_year)

In [ ]:
# Atmospheric SSP data is freely available via ESGF
download_atmospheric_variables_data(data_subdirectory, atmospheric_variables, SSPs)

## PROCESS

In [ ]:
process_netcdf_data(data_subdirectory, output_subdirectory, 
                    country_shapes_filepath, country_codes_filepath, alignments_filepath, 
                    frequency)

Processing files...
ssp1_ch4.nc


Aggregating:  67%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                                                           | 173/258 [24:37<11:38,  8.21s/it]

In [ ]:
# Country-level GDP and population SSP data is downloaded from IIASA via a registered account (free)

# Process the socioeconomic datasets
extract_socioeconomic_ssp_data(socioeconomic_ssp_filepath, output_subdirectory, socioeconomic_variables, SSPs, start_year, end_year, frequency)
process_population_historical_data(population_historical_filepath, output_subdirectory, frequency)
process_GDP_historical_data(gdp_historical_filepath, output_subdirectory, frequency)

# Scratch

In [ ]:
# CH4
ds = xr.open_dataset("/Users/michellewan/PycharmProjects/TraCE/data_pipeline/raw/ch4/ssp1_ch4.nc")

In [ ]:
ds = ds.sel(lev=ds["lev"].max().item())
ds = ds.drop("lev")

In [ ]:
country_shapes = gpd.read_file(country_shapes_filepath)

In [ ]:
ds["ch4"]

In [ ]:
agg_ds = aggregate_country_data(ds, country_shapes)

In [ ]:
df = convert_nc_to_csv(agg_ds, "ch4")

In [ ]:
# Extract the necessary data variables
time_steps = agg_ds.time.values
countries = agg_ds.country.values
variable = agg_ds.values

# Reshape the data into a 2D array
# Rows: time steps, Columns: countries
variable_reshaped = variable.transpose(1, 0)

# Create a pandas DataFrame
df = pd.DataFrame(data=variable_reshaped, index=time_steps, columns=countries)

In [ ]:
df.index

In [ ]:
df.index = df.index.astype(str)

In [ ]:
df.index

In [ ]:
import cftime

# Convert the cftime.DatetimeNoLeap objects to standard datetime objects
datetime_index = [cftime.datetime(date.year, date.month, date.day) for date in df.index]


In [ ]:
df.index = pd.to_datetime(df.index.astype(str))
df.resample("Q").mean()

In [7]:
df.index

NameError: name 'df' is not defined